<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🟢 תהליך הסבת נקודות גבול החלקות לבנק"ל מודרני 🟢

In [ ]:
print('2. Parcel border points:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
קריאת נתונים

In [ ]:
print('  2.a Reading inputs')

In [ ]:
RawData_folder = r"\\mapi_shares\MNCDB\צוות מידע\RawData"

<h1 style="text-align:right;font-size:100%">
שכבת נקודות גבול

In [ ]:
coverage_points_cols = ['PARCEL_SDE_', 'POINT_NAME', 'CONTROL_ID', 'REG_STATUS', 'POINT_MARK', 'SOURCE_CODE', 'geometry']
coverage_df = gpd.read_file(RawData_folder + '\ArcStorm\PARCEL_NODE.gdb', layer='PARCEL_NODE', include_fields=coverage_points_cols)

del coverage_points_cols

print('         input 1/6 read')

<h1 style="text-align:right;font-size:100%">
שכבת גושים מודרנית

In [ ]:
today = dt.date.today()
date_name = str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name
blocks_df = gpd.read_file(product_path, layer='Blocks', include_fields=['SetteledDate', 'geometry'])

del [today, date_name, product_path]

print('         input 2/6 read')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר

In [ ]:
talar_borders = gpd.read_file(RawData_folder + "\CADSDE\\talar_borders.gdb", include_fields=['STATUS', 'SURVEY_DATE','geometry'])

print('         input 3/6 read')

<h1 style="text-align:right;font-size:100%">
שכבת נקודות גבול בתהליך

In [ ]:
points_in_process_path = RawData_folder + "\CADSDE\\Points_InProcess.gdb"
points_in_process_cols = ['POINT_NAME', 'ACCURACY', 'MARK_CODE', 'SOURCE_CODE', 'INSERTDATE', 'geometry']
points_in_process_df =  gpd.read_file(points_in_process_path, layer='Points_InProcess', include_fields=points_in_process_cols)
points_in_process_df.columns = points_in_process_df.columns.str.lower()

del [points_in_process_path, points_in_process_cols]

print('         input 4/6 read')

<h1 style="text-align:right;font-size:100%">
שכבת גושי קמ"ק

In [ ]:
cbc_path = RawData_folder + "\Local\CadstreBasedCoordBlocks.gpkg"
cbc_blocks = gpd.read_file(cbc_path, layer='cbc_blocks', include_fields=['geometry'])
del cbc_path
print('         input 5/6 read')

<h1 style="text-align:right;font-size:100%">
שכבת גושי ים תיכון

In [ ]:
medeteraninan_path = RawData_folder + "\Local\medeteraninanBlocks.gpkg"
medeteraninanBlocks = gpd.read_file(medeteraninan_path, layer='medeteraninanBlocks', include_fields=['geometry'])
del medeteraninan_path
print('         input 6/6 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים

In [ ]:
print('  2.b Cleaning data')

<h1 style="text-align:right;font-size:125%">
יצירת שדה טקסט של גיאומטרית הנקודות לצורך מיון/סינון/מיזוג בהמשך

In [ ]:
coverage_df['geom_str'] = coverage_df['geometry'].x.astype(str) + ',' + coverage_df['geometry'].y.astype(str)

points_in_process_df['geom_str'] = points_in_process_df['geometry'].x.astype(str) + ',' + points_in_process_df['geometry'].y.astype(str)

<h1 style="text-align:right;font-size:125%"> 
הפקת שדות של שנים עבור שדה רשת הבקרה

<h1 style="text-align:right;font-size:110%">
:שנת הסדר הגוש
<h1 style="text-align:right;font-size:100%"> 
<h1 style="text-align:right;font-size:100%"> 
הוספת שדה של שנת הסדר לתוצר גושים
<h1 style="text-align:right;font-size:100%">
צירוף שנת הסדר לנקודות גבול
<h1 style="text-align:right;font-size:100%">
סידור ערכים לפי שנת ההסדר
<h1 style="text-align:right;font-size:100%">
מחיקת גיאומטריות כפולות, שמירת רשומה בעלת שנת הסדר אחרונה

In [ ]:
blocks_df['setteledYear'] = pd.to_datetime(blocks_df['SetteledDate']).dt.year.fillna(0).astype(int)

setteld_points_df = coverage_df.sjoin(blocks_df[['setteledYear', 'geometry']], how='inner', predicate='intersects')
setteld_points_df.sort_values(['geom_str', 'setteledYear'], inplace=True)
setteld_points_df.drop_duplicates(subset='geom_str', keep='last', ignore_index=True, inplace=True)
setteld_points_df = setteld_points_df[['geom_str', 'setteledYear']]

del blocks_df

<h1 style="text-align:right;font-size:110%">
:שנת מדידה בתצ"ר
<h1 style="text-align:right;font-size:100%">
<h1 style="text-align:right;font-size:100%">
סינון לרשומות תצ"ר שכשרות לרישום או סופיות
<h1 style="text-align:right;font-size:100%">
קריאת שדה תאריך מדידה כסוג תאריך ומילוי ערכים חסרים
<h1 style="text-align:right;font-size:100%">
הרחבת גבולות התצ"ר ב-2 מטרים
<h1 style="text-align:right;font-size:100%">
יצירת שכבת נקודות שנמצאות בתצ"ר מורחבים
<h1 style="text-align:right;font-size:100%">
סידור הערכים לפי מספר זיהוי של נקודת גבול ותאריך מדידה
<h1 style="text-align:right;font-size:100%">
מחיקת נקודות גבול כפולות , שמירת רשומה בעלת תאריך מדידה אחרון
<h1 style="text-align:right;font-size:100%">
יצירת שדה של שנת מדידה

In [ ]:
talar_borders = talar_borders[talar_borders['STATUS'].isin([3, 4])]
talar_borders['SURVEY_DATE'] = pd.to_datetime(talar_borders['SURVEY_DATE']).fillna('01/01/1900')
talar_borders['geometry'] = talar_borders['geometry'].buffer(2)
talar_points = coverage_df.sjoin(talar_borders, how='inner', predicate='intersects')
talar_points.sort_values(['PARCEL_SDE_','SURVEY_DATE'], inplace=True)
talar_points.drop_duplicates(subset='PARCEL_SDE_', keep='last', inplace=True)
talar_points['surveyYear'] = talar_points['SURVEY_DATE'].dt.year.astype(int)
talar_points = talar_points[['geom_str', 'surveyYear']]

<h1 style="text-align:right;font-size:110%">
:תאריך יצירה בבנק"ל של נקודות בתהליך
<h1 style="text-align:right;font-size:100%">
<h1 style="text-align:right;font-size:100%">
יצירת שדה של שנת יצירה
<h1 style="text-align:right;font-size:100%">
הסרת שדה תאריך יצירה ישן
<h1 style="text-align:right;font-size:100%">
סידור הערכים לפי שנת יצירה
<h1 style="text-align:right;font-size:100%">
מחיקת נקודות גבול בתהליך כפולות, שמירת רשומה בעלת תאריך יצירה אחרון

In [ ]:
points_in_process_df['insertyear'] = points_in_process_df['insertdate'].astype('datetime64').dt.year
points_in_process_df.drop(columns='insertdate', inplace=True)
points_in_process_df.sort_values(['geom_str', 'insertyear'], inplace=True)
points_in_process_df.drop_duplicates(subset='geom_str', keep='last', ignore_index=True, inplace=True)

<h1 style="text-align:right;font-size:125%">
:הגדרה ומיון של סיווג נקודות גבול
<h1 style="text-align:right;font-size:100%">
בשלב זה ישנם 2 מקורות לנתונים: נקודות גבול בתהליך, נקודות גבול של חלקות כאשר לכל מקור קיימת שכבת נקודות קמ"ק שבהן יתקבל סיווג לערך 13
<h1 style="text-align:right;font-size:100%">
<h1 style="text-align:right;font-size:125%">
א) נקודות גבול בתהליך
<h1 style="text-align:right;font-size:100%">
יצירת שכבת נקודות גבול בתהליך שנמצאים בתוך או על גבול גושי קמ"ק
<h1 style="text-align:right;font-size:100%">
הסרת כפילויות
<h1 style="text-align:right;font-size:100%">
מיפוי ערכי סייוג נקודה התנליך לפי אפיון
<h1 style="text-align:right;font-size:100%">
יצירת שדה בוליאני לזיוהי היאם הנקודה בקמ"ק (1) או לא (0)
<h1 style="text-align:right;font-size:100%">
החלפת ערכי סיווג ל-13 לכל נקודה בתהליך שנמצאת גם בקמ"ק    
<h1 style="text-align:right;font-size:100%">
נקודות בגושי ים תיכון יקבלו את הסיוג 1    

In [ ]:
cbc_points_in_process = points_in_process_df.sjoin(cbc_blocks[['geometry']], predicate = 'intersects')
cbc_points_in_process.drop_duplicates(subset=['geom_str'], inplace=True)

In [ ]:
process_accuracy_clf = {0 : 24,
                        1 : 1,
                        2 : 12,
                        3 : 13,
                        4 : 13,
                        5 : 13,
                        6 : 24,
                        7 : 24,
                        8 : 24,
                        9 : 24,
                        12 : 12,
                        13 : 13,
                        24 : 24,
                        None : 24
                       }

points_in_process_df['process_accuracy'] = points_in_process_df['accuracy'].fillna(0).astype('int64').map(process_accuracy_clf)

In [ ]:
points_in_process_df['CBC'] = np.where(points_in_process_df['geom_str'].isin(cbc_points_in_process['geom_str']), 1, 0)
points_in_process_df['process_accuracy'] = np.where(points_in_process_df['CBC']==1, 13, points_in_process_df['process_accuracy'])
points_in_process_df.drop(columns='CBC', inplace=True)

In [ ]:
medeteraninan_in_process = points_in_process_df.sjoin(medeteraninanBlocks[['geometry']], predicate = 'intersects')
medeteraninan_in_process.drop_duplicates(inplace=True, subset=['geom_str'])
points_in_process_df['medeteraninan'] = np.where(points_in_process_df['geom_str'].isin(medeteraninan_in_process['geom_str']), 1, 0)
points_in_process_df['process_accuracy'] = np.where(points_in_process_df['medeteraninan']==1, 1, points_in_process_df['process_accuracy'])
points_in_process_df.drop(columns='medeteraninan', inplace=True)

<h1 style="text-align:right;font-size:125%">
ב) נקודות גבול חלקות קיימות
<h1 style="text-align:right;font-size:100%">
תהליך זהה לסעיף קודם בהתאם לנקודות גבול חלקות

In [ ]:
cbc_coverage = coverage_df.sjoin(cbc_blocks[['geometry']], predicate = 'intersects')
cbc_coverage.drop_duplicates(inplace=True, subset='geom_str')

In [ ]:
coverage_accuracy_clf = {0 : 24,
                         1 : 24,
                         2 : 24,
                         3 : 24,
                         4 : 24,
                         5 : 24,
                         6 : 13,
                         7 : 24,
                         33 : 24,
                         None : 24
                        }

coverage_df['coverage_accuracy'] = coverage_df['REG_STATUS'].fillna(0).astype('int64').map(coverage_accuracy_clf)

In [ ]:
coverage_df['CBC'] = np.where(coverage_df['geom_str'].isin(cbc_coverage['geom_str']), 1, 0)
coverage_df['coverage_accuracy'] = np.where(coverage_df['CBC']==1, 13, coverage_df['coverage_accuracy'])
coverage_df.drop(columns='CBC', inplace=True)

In [ ]:
medeteraninan_coverage = coverage_df.sjoin(medeteraninanBlocks[['geometry']], predicate = 'intersects')
medeteraninan_coverage.drop_duplicates(inplace=True, subset=['geom_str'])
coverage_df['medeteraninan'] = np.where(coverage_df['geom_str'].isin(medeteraninan_coverage['geom_str']), 1, 0)
coverage_df['coverage_accuracy'] = np.where(coverage_df['medeteraninan']==1, 1, coverage_df['coverage_accuracy'])
coverage_df.drop(columns='medeteraninan', inplace=True)

<h1 style="text-align:right;font-size:100%">
הסרת משתנים להורדת זיכרון

In [ ]:
del [cbc_blocks, medeteraninanBlocks, cbc_points_in_process, medeteraninan_in_process, cbc_coverage, medeteraninan_coverage]

<h1 style="text-align:right;font-size:125%">
מיזוג שכבות של נקודות בתהליך ונקודת גבול חלקות

In [ ]:
print('  2.c Data conversion')

<h1 style="text-align:right;font-size:100%">
מיזוג לפי שדה טקסטי של גיאומטריה
<h1 style="text-align:right;font-size:100%">
מיזוג נלווה של שדות תאריכי שנת הסדר, שנת מדידה
<h1 style="text-align:right;font-size:100%">
יצירת שדה של שנת רשת בקרה לפי סדר עדיפות
<h1 style="text-align:right;font-size:100%">
הסרת שדות קודמים של שנים לרשת בקרה
<h1 style="text-align:right;font-size:100%">
הסרת נקודות כפולות שנוצרו לאחר המיזוגים

In [ ]:
merged = coverage_df.merge(points_in_process_df, on='geom_str', how='left', suffixes=(None, 'process'))
merged = merged.merge(setteld_points_df[['geom_str', 'setteledYear']], on='geom_str', how='left', suffixes=(None, 'setteled'))
merged = merged.merge(talar_points[['geom_str', 'surveyYear']], on='geom_str', how='left', suffixes=(None, 'survey'))

merged['geoYear'] = merged['setteledYear'].fillna(merged['surveyYear']).fillna(merged['insertyear']).fillna(0)
merged.drop(columns=['setteledYear', 'insertyear','surveyYear'], inplace=True)
merged.sort_values(['geom_str', 'geoYear'], inplace=True)
merged.drop_duplicates(subset='geom_str', keep='last', ignore_index=True, inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
יצירת שכבה חדשה בהתאם לאפיון

In [ ]:
print('  2.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("PointUniqueID", int),
                        ("PointName", str),
                        ("IsControlBorder", int),
                        ("Class", int),
                        ("SourceCode", int),
                        ("MarkCode", int),
                        ("GeodeticNetwork", int),
                        ("Is3D", int),
                        ("Connected3DParcelsIds", int)
                        ])


BorderPoints = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                                geometry = None,
                                crs = 2039)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
איכלוס שדות

In [ ]:
BorderPoints['PointUniqueID'] = merged['PARCEL_SDE_']


BorderPoints['PointName'] = merged['POINT_NAME'].astype(str)


BorderPoints['IsControlBorder'] = merged['CONTROL_ID'].apply(lambda x: 0 if x == 0 else 1).astype('int64')


BorderPoints['Class'] = merged['process_accuracy'].fillna(merged['coverage_accuracy']).astype('int64')


source_clf = {  -2 : 0,
                 0 : 0,
                 1 : 0,
                 2 : 0,
                 3 : 0,
                 4 : 0,
                 5 : 5,
                 6 : 5,
                 7 : 5,
                 8 : 8,
                 9 : 9,
                 10 : 10,
                 11 : 11,
                 12 : 11,
                 13 : 13,
                 14 : 14,
                 15 : 14,
                 16 : 16,
                 17 : 17,
                 18 : 18,
                 None : 0
                }
BorderPoints['SourceCode'] = merged['source_code'].fillna(merged['SOURCE_CODE']).astype('int64').map(source_clf)


mark_clf = {  0 : 0,
              1 : 1,
              2 : 2,
              3 : 3,
              4 : 4,
              5 : 5,
              6 : 6,
              7 : 7,
              8 : 8,
              9 : 9,
              10 : 10,
              11 : 11,
              12 : 12,
              13 : 13,
              14 : 14,
              15 : 0,
              16 : 0,
              17 : 0,
              20 : 20,
              39 : 0,
              50 : 0,
              54 : 0,
              94 : 0,
              None : 0
           }
BorderPoints['MarkCode'] = merged['mark_code'].fillna(merged['POINT_MARK']).astype('int64').map(mark_clf)


geodetic_clf = { range(0, 1994) : 0,
                 range(1994, 1999) : 1,
                 range(1999, 2009) : 2,
                 range(2009, 9999) : 3
               }
BorderPoints['GeodeticNetwork'] = merged['geoYear'].apply(lambda x: next((v for k, v in geodetic_clf.items() if x in k), 0))


BorderPoints['Is3D'] = 0


BorderPoints['Connected3DParcelsIds'] = None


BorderPoints['geometry'] = merged['geometry']

In [ ]:
BorderPoints.to_crs(2039, inplace=True)

In [ ]:
del [merged, coverage_df, points_in_process_df]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
שמירת תוצאות

In [ ]:
print('  2.e Exporting results to product folder')

<h1 style="text-align:right;font-size:100%">
בחירת נתיב
<h1 style="text-align:right;font-size:100%">
שמירה כטבלה
<h1 style="text-align:right;font-size:100%">
שמירה כשכבה גיאוגרפית

In [ ]:
#csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\BorderPoints.csv'
json_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\BorderPoints.geojson'


#BorderPoints.drop(columns='geometry').to_csv(csv_dir ,encoding='utf8', index=False)
BorderPoints.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\BorderPoints-Save as Feature Class.bat"]);